<a href="https://colab.research.google.com/github/jeremyindelicato/Orapi_LLM_fire-tune/blob/main/OrapIA_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1ère étape : Installation des bibliothèques nécessaires 🤗

In [ ]:
!pip uninstall -y transformers datasets
!pip install transformers datasets

2ème étape : Importation des bibliothèques 📚

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForQuestionAnswering, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import Trainer

3ème étape :  Chargement et Préparation des Données 📁

In [ ]:
data = pd.read_csv('/content/data_set_products_clean_ORAPI_M3.csv', sep=';', on_bad_lines='skip', engine="python")

print(data.head())

# Convertir le DataFrame pandas en un Dataset Hugging Face
dataset = Dataset.from_pandas(data)

4ème étape: Tokenisation des Données ⚗️

In [ ]:
# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

dataset = Dataset.from_pandas(data)  # Convert the DataFrame to a Dataset object

# Fonction de tokenisation
def tokenize_function(examples):
    return tokenizer(examples['Fonction du produit francais'], padding='max_length', truncation=True)

# Appliquer la tokenisation au dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

5ème étape : Chargement du modèle pré-entraîné 🤖

In [ ]:
# Charger un modèle BERT pré-entraîné pour la tâche de question-réponse
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


6ème étape : Configuration des paramètres d'entraînement 🏋️

In [ ]:
# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

7ème étape : Création & Initialisation du trainer 🤏

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Calcule la perte pour la tâche de question-réponse.
        """
        # Effectuer un passage avant à travers le modèle
        outputs = model(**inputs)

        # Extraire les logits de début et de fin
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

        # Extraire les positions de début et de fin des labels
        start_positions = inputs.pop("start_positions")
        end_positions = inputs.pop("end_positions")

        # Calculer la perte de question-réponse en utilisant CrossEntropyLoss
        loss_fct = torch.nn.CrossEntropyLoss()
        start_loss = loss_fct(start_logits.view(-1, start_logits.size(-1)), start_positions.view(-1))
        end_loss = loss_fct(end_logits.view(-1, end_logits.size(-1)), end_positions.view(-1))
        total_loss = (start_loss + end_loss) / 2

        return (total_loss, outputs) if return_outputs else total_loss

# Instanciation du CustomTrainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

8ème étape : Lancement de l'entrainement 🏁

In [ ]:
trainer.train()

9ème étape : Évaluation du modèle ⭐

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

10ème étape : Sauvegarde du modèle 💾

In [ ]:
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

11ème étape : Utilisation du modèle pour répondre aux questions 💬

In [ ]:
# Fonction pour poser une question au modèle
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Obtenir la réponse
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

# Exemple d'utilisation
context = data['Fonction du produit francais'][0]  # Prenez une description de produit
question = "Quel produit est adapté pour le nettoyage des machines industrielles ?"
print(answer_question(question, context))